In [1]:
import pandas as pd

In [26]:
df = pd.read_csv('furniture_amazon_dataset.csv')

# Get header
header = df.columns.tolist()

# Get rows as list of dicts or list of lists
rows = df.values.tolist()  # or df.to_dict(orient='records')

print("Header:", header)
print("First row:", rows[0])

Header: ['asin', 'url', 'title', 'brand', 'price', 'availability', 'categories', 'primary_image', 'images', 'upc', 'manufacturer', 'item_model_number', 'package_dimensions', 'date_first_available', 'country_of_origin', 'color', 'material', 'style', 'important_information', 'product_overview', 'about_item', 'description', 'specifications', 'uniq_id', 'scraped_at']
First row: ['B0CJHKVG6P', 'https://www.amazon.com/dp/B0CJHKVG6P', 'GOYMFK 1pc Free Standing Shoe Rack, Multi-layer Metal Shoe Cap Rack With 8 Double Hooks For Living Room, Bathroom, Hallway', 'GOYMFK', '$24.99', 'Only 13 left in stock - order soon.', "['Home & Kitchen', 'Storage & Organization', 'Clothing & Closet Storage', 'Shoe Organizers', 'Free Standing Shoe Racks']", 'https://m.media-amazon.com/images/I/416WaLx10jL._SS522_.jpg', "['https://m.media-amazon.com/images/I/416WaLx10jL._SS522_.jpg ', ' https://m.media-amazon.com/images/I/41kuxipTsuL._SS522_.jpg ', ' https://m.media-amazon.com/images/I/51T9x4yZd3L._SS522_.jpg ', 

In [27]:
import ast
def get_list_from_string(string_list):
    items = ast.literal_eval(string_list)
    return [n.strip() for n in items]
    

In [28]:
categories_from_data = []
for row in rows:
    categories_from_data.extend(get_list_from_string(row[6]))
set(categories_from_data)

{'Accent Furniture',
 'Accessories',
 'Adirondack Chairs',
 'Baby Products',
 'Bar & Serving Carts',
 'Bar Cabinets',
 'Bars & Wine Cabinets',
 'Barstools',
 'Bassinets',
 'Bath',
 'Bathroom Accessories',
 'Bathroom Furniture',
 'Bathroom Hardware',
 'Bathroom Mirrors',
 'Bathroom Sets',
 'Bean Bags',
 'Bean Bags, Covers & Refills',
 'Beauty & Personal Care',
 'Bed Frames',
 'Bed Parts',
 'Bedding',
 'Bedroom Armoires',
 'Bedroom Furniture',
 'Beds',
 'Beds, Frames & Bases',
 'Bedside Cribs',
 'Bookcases',
 'Bookcases, Cabinets & Shelves',
 'Boot & Shoe Boxes',
 'Box Springs',
 'Chairs',
 'Chairs & Seats',
 'Chairs & Sofas',
 'Changing & Dressing',
 'Chests & Dressers',
 'Clothing & Closet Storage',
 'Coat Racks',
 'Coffee Tables',
 'Computer Gaming Chairs',
 'DVD Cases',
 'Desk Chairs',
 'Dining Room Furniture',
 'Doormats',
 'Dressers',
 'Electronics',
 'End Tables',
 'Entryway Furniture',
 'Filing Products',
 'Folding Chairs',
 'Folding Shoe Racks',
 'Folding Tables',
 'Folding Tabl

In [29]:
import json

# Open the JSON file
with open('categories.json', encoding='utf-8') as f:
    categories = json.load(f)

categories_dict_list = [{'pk': x['pk'], 'name': x['fields']['name']} for x in categories]
categories_dict_list

[{'pk': 1, 'name': 'Event Appropriate Furniture'},
 {'pk': 2, 'name': 'Chairs'},
 {'pk': 3, 'name': 'Chairs & Sofas'},
 {'pk': 4, 'name': 'Folding Chairs'},
 {'pk': 5, 'name': 'Folding Tables'},
 {'pk': 6, 'name': 'Folding Tables & Chairs'},
 {'pk': 7, 'name': 'Ottomans'},
 {'pk': 8, 'name': 'Poufs'},
 {'pk': 9, 'name': 'Sofas & Couches'},
 {'pk': 10, 'name': 'Table & Chair Sets'},
 {'pk': 11, 'name': 'Tables'},
 {'pk': 12, 'name': 'Tables & Chairs'},
 {'pk': 13, 'name': 'Nesting Tables'},
 {'pk': 14, 'name': 'TV Trays'},
 {'pk': 15, 'name': 'Outdoor & Patio'},
 {'pk': 16, 'name': 'Patio Furniture & Accessories'},
 {'pk': 17, 'name': 'Patio Seating'},
 {'pk': 18, 'name': 'Bar & Entertainment'},
 {'pk': 19, 'name': 'Bar & Serving Carts'},
 {'pk': 20, 'name': 'Bars & Wine Cabinets'},
 {'pk': 21, 'name': 'Freestanding Wine Racks & Cabinets'},
 {'pk': 22, 'name': 'Wine Cabinets'},
 {'pk': 23, 'name': 'Wine Racks & Cabinets'},
 {'pk': 24, 'name': 'Bar Cabinets'},
 {'pk': 25, 'name': 'Barsto

In [30]:
import re
def get_clean_string(phrase):
    if type(phrase) is str:
        # Step 1: Remove escape characters (like \' and \")
        cleaned_string = re.sub(r"[\\\'\"][\\\'\"]", "", phrase)
        
        # Step 2: Remove unwanted characters (like 【, 】, etc.)
        cleaned_string = re.sub(r"[【】]", "", cleaned_string)
        
        # Step 3: Remove leading and trailing spaces, if any
        cleaned_string = cleaned_string.strip()
        
        # Step 4: Remove extra spaces between words (if needed)
        cleaned_string = re.sub(r'\s+', ' ', cleaned_string)

        cleaned_string = cleaned_string.replace("'", "").replace("[", "").replace("]", "")
        
        return cleaned_string
        
    return ''


In [31]:
def get_category(categories_string, categories_dec):
    categories_list = get_list_from_string(categories_string)
    return [ d for d in categories_dec if d['name'] in categories_list ]

In [37]:
import random

supplies = []
supply_images = []
quantities = [15, 20, 40, 50, 100, 150, 200, 300]
pk = 1
images_pk = 1
for row in rows:
    item_categories = get_category(row[6],categories_dict_list)
    category = item_categories[0]['pk'] if len(item_categories) else None 
    if category is not None:
        price =  int(float(row[4].replace('$',''))) * 20 / 100 if type(row[4]) is str else 10 
        description = get_clean_string(row[20])
        supplies.append({'pk' : pk, 
                         "model": "supplies.supply",
                         "fields": {
                             'name': row[2],
                             'sku': row[0],
                             'category': category,
                             'description': description,
                             'quantity_available': random.choice(quantities),
                             'price_per_day': price
                         }
                        })
        images_list = get_list_from_string(row[8])
        for image in images_list:
            supply_images.append({ 'pk': images_pk,
                                   "model": "supplies.supplyimage",
                                   "fields": {
                                       'supply': pk,
                                       'image_url': image
                                   }
                                 })
            images_pk += 1
                                   
                                   
        pk += 1
                         

print(len(supplies))
print(len(supply_images))

143
930


In [38]:
supplies[0]

{'pk': 1,
 'model': 'supplies.supply',
 'fields': {'name': 'subrtex Leather ding Room, Dining Chairs Set of 2, Black',
  'sku': 'B0B66QHB23',
  'category': 2,
  'description': 'Easy Assembly: Set of 2 dining room chairs. To make it easier for you to install our chairs, we have included tools and instructions in the package and added an installation video to the webpage. It will make the whole installation process easier and quicker(Estimated time 10-20 minutes). ,  Waterproof Leather Fabric: subrtex upholstered dinging room chair is made of premium pu leather, waterproof, breathable, skin-friendly and easy to clean when it gets water or dirty. ,  Black Rubber Wood Legs: Hold up to 265 LBS. Anti-slip padded feet design that can prevent the floor from scratching. Tips: The solid wooden legs can not be replaced individually, if you encounter problems with the legs, you can contact us to replace a new dining chair. ,  Ergonomic Backrest Design: Ergonomically designed chair back, effectivel

In [39]:
supply_images[0:5]

[{'pk': 1,
  'model': 'supplies.supplyimage',
  'fields': {'supply': 1,
   'image_url': 'https://m.media-amazon.com/images/I/31SejUEWY7L._SS522_.jpg'}},
 {'pk': 2,
  'model': 'supplies.supplyimage',
  'fields': {'supply': 1,
   'image_url': 'https://m.media-amazon.com/images/I/41mr+A9JmbL._SS522_.jpg'}},
 {'pk': 3,
  'model': 'supplies.supplyimage',
  'fields': {'supply': 1,
   'image_url': 'https://m.media-amazon.com/images/I/41JjrWgA0XL._SS522_.jpg'}},
 {'pk': 4,
  'model': 'supplies.supplyimage',
  'fields': {'supply': 1,
   'image_url': 'https://m.media-amazon.com/images/I/41ie5FbnfkL._SS522_.jpg'}},
 {'pk': 5,
  'model': 'supplies.supplyimage',
  'fields': {'supply': 1,
   'image_url': 'https://m.media-amazon.com/images/I/41EDoJzFehL._SS522_.jpg'}}]

In [40]:
with open('supplies.json', 'w') as fout:
    json.dump(supplies , fout)

In [42]:
with open('supply_images.json', 'w') as fout:
    json.dump(supply_images , fout)